In [1]:
# 02/01/2021 Luís

# This notebook is similar to generate_MR_uniaxial and generate_MR_equibiaxial.

# In this case, we consider a load applied in the xx direction, causing a stretch in this direction,
# while stretching is restricted in the yy direction (stretch_yy = 1).
# There are no restrictions on the zz direction, but we will assume incompressibility, so stretch_zz = 1/stretch,
# because the Jacobian must equal 1.

# Boundary conditions:
# szz = 0


# ON GOING:
# I will try to account for compressibility.
# Will start trying some other models from this source: https://doi.org/10.1002/msd2.12013

In [2]:
import pandas as pd
import random
import sympy as sym
import numpy as np
from matplotlib import pyplot as plt

pd.set_option('display.max_rows', None)

In [3]:
def mooney_rivlin_shear(params, stretch):
    #returns cauchy stress at xx direction
    
    # Right Cauchy-Green Deformation Tensor 'C'
    C11 = sym.Symbol('C11')
    C12 = sym.Symbol('C12')
    C13 = sym.Symbol('C13')
    C21 = sym.Symbol('C21')
    C22 = sym.Symbol('C22')
    C23 = sym.Symbol('C23')
    C31 = sym.Symbol('C31')
    C32 = sym.Symbol('C32')
    C33 = sym.Symbol('C33')
    C = sym.Matrix([[C11,C12,C13], [C21,C22,C23], [C31,C32,C33]])  
    
    # Compute the invariants I1 and I2 of the tensor C
    i1=sym.trace(C)
    i2 = C11*C22 + C22*C33 + C33*C11 - C12**2 - C23**2 - C31**2
    
    #Generate SEF (Strain Energy Function)
    #symbolic C10 and C01
    c10s = sym.Symbol('c10s')
    c01s = sym.Symbol('c01s')
    sef=c10s*(i1-3) + c01s*(i2-3)
    #Second Piola Kirchoff Stresses
    S11=2*sym.diff(sef,C11)
    S12=2*sym.diff(sef,C12)
    S13=2*sym.diff(sef,C13)
    S21=2*sym.diff(sef,C21)
    S22=2*sym.diff(sef,C22)
    S23=2*sym.diff(sef,C23)
    S31=2*sym.diff(sef,C31)
    S32=2*sym.diff(sef,C32)
    S33=2*sym.diff(sef,C33)
    S = sym.Matrix([[S11,S12,S13], [S21,S22,S23], [S31,S32,S33]])
    
    # Deformation Gradient assuming incompressibility and the load described in the beggining of the notebook.
    F=sym.Matrix([[stretch,0,0], [0,1,0], [0,0,1/stretch]])
    Ft=sym.transpose(F)
    C=Ft*F
    Jac=sym.det(F) # is allways 1 because we have assumed incompressibility
    C11v=C[0,0]
    C12v=C[0,1]
    C13v=C[0,2]
    C21v=C[1,0]
    C22v=C[1,1]
    C23v=C[1,2]
    C31v=C[2,0]
    C32v=C[2,1]
    C33v=C[2,2]
    
    cauchy=(1/Jac)*(F*S*Ft) # push-forward. cauchy stresses with no BCs
    stress=cauchy[0,0]-cauchy[2,2] # imposing of boundary conditions
    stress_abq=sym.Matrix([[stress,0,0], [0, stress,0], [0,0,0]]) # stress11 == stress22; stress33 == 0
    s11_val=stress_abq.subs([(C11, C11v), (C12, C12v), 
                                  (C13, C13v),(C21, C21v), 
                                  (C22, C22v), (C23, C23v),
                                  (C31, C31v), (C32, C32v), 
                                  (C33, C33v),(c10s,params[0]), (c01s,params[1])])
    
    return stretch,s11_val[0,0]

def get_curve(params,stretch_min,stretch_max,ninc):
    #stores Mooney-Rivlin equibiaxial runs between minimum and a maximum stretch
    lst=[mooney_rivlin_shear(params, stretch) for stretch in np.linspace(stretch_min,stretch_max,ninc)]
    return lst

In [4]:
# Let us aplly our function to a simbolic stretch and simbolic c10 and c01.

c10 = sym.Symbol('c10')
c01 = sym.Symbol('c01')
l = sym.Symbol('l') # Symbolic Stretch
[stretch, stress] = mooney_rivlin_shear([c10,c01], l)
sym.simplify(stress)

# Cauchy stress as a function of principal stretch (l or lambda) and parameters c10, c01.
# This is in conformity with results found in litterature for incompressible, equibiaxial loading.
# For example: https://doi.org/10.1002/msd2.12013 Section 2.1.3.

2*(c01*l**4 - c01 + c10*l**4 - c10)/l**2

In [5]:
#Initial data-----------------------------------------------------------------------------------------------------

C10min=0.1     #min C10 value
C10max=10.0    #max C10 value
C01min=0.1     #min C01 value
C01max=2.0     #max C01 value
decimals=2   #number of decimal cases of C10 and C01 variables

n_c10=5               #number of C10 variables
n_c01=5               #number of C01 variables
n = n_c10 * n_c01     #number of combinations
#
st_max=3.0             #applied stretch
st_min=1.0
ninc=100        #number of stretch increments
#------------------------------------------------------------------------------------------------------------------

In [6]:
# Generates random values for c10 and c01
c10_list = np.round(np.random.uniform(C10min, C10max, size=(n_c10, 1)), decimals)
c01_list = np.round(np.random.uniform(C01min, C01max, size=(n_c01, 1)), decimals)
params = np.array([])

# Computes the n combinations of c10 and c01 values
for i in c10_list:
    for j in c01_list:
        params = np.append(params, np.array([i,j]))
params = params.reshape(-1, 2)

In [7]:
# Append the combinations to a DataFrame

df = pd.DataFrame(params[:,0], columns = ['c10'])
df['c01'] = params[:,1]
df

# Later I will deal with duplicates

,c10,c01
0,3.02,1.17
1,3.02,0.49
2,3.02,0.21
3,3.02,0.14
4,3.02,0.33
5,9.97,1.17
6,9.97,0.49
7,9.97,0.21
8,9.97,0.14
9,9.97,0.33


In [ ]:
#generate (x,y) data for each unique c10 at the dataframe
df['data']=df.apply(lambda x : get_curve(x,st_min,st_max,ninc), axis=1)
df

In [ ]:
example = np.array(df.data[15])

In [ ]:
y = example[:, 1]
x = example[:, 0]
plt.plot(x,y)

In [ ]:
df.to_pickle('./data.pkl')

In [ ]:
# Just to manually check that the lambda function worked properly
check = np.array(get_curve([6.52,1.23],st_min,st_max,ninc))
plt.plot(check[:,0],check[:,1])